In [448]:
import numpy as np
import itertools

In [449]:
i1 = 0.05
i2 = 0.10

b1 = 0.35

w1 = 0.15
w2 = 0.20
w3 = 0.25
w4 = 0.30

o1 = 0.01
o2 = 0.99

b2 = 0.60

w5 = 0.4
w6 = 0.45
w7 = 0.5
w8 = 0.55

In [450]:
dimensions = [2,2,2]

In [451]:
def calculate_weight_set_dimensions(dimensions):

    a, b = itertools.tee(dimensions[::-1])
    next(b, None)
    weight_set_dimensions = list(zip(a, b))[::-1]
    
    return weight_set_dimensions

In [452]:
calculate_weight_set_dimensions(dimensions)

[(2, 2), (2, 2)]

In [453]:
def initialise_weights(dimensions):
    
    # For single hidden layer neural network there will be 2 sets of weights;
    # 1- one set to hidden layer
    # 2- one set from hidden layer
    # number of weight sets = no_of_hidden_layers + 1
    
    weight_dims = calculate_weight_set_dimensions(dimensions)
    no_of_weight_sets = len(weight_dims)
    
    # W_set holds weight sets such as w1, w2, w3 etc.
    W = np.empty_like(range(no_of_weight_sets), dtype=object)
    for index, (row, column) in enumerate(weight_dims):
         W[index] = np.random.rand(row, column)
            
    return W

In [454]:
def forwardpropagation(input_vals, W):
    
    weight_dims = calculate_weight_set_dimensions(dimensions)
    no_of_weight_sets = len(weight_dims)
    
    Z = np.empty_like(range(no_of_weight_sets + 1), dtype=object)
    Z[0] = input_vals
    for index in range(no_of_weight_sets):
        Z[index + 1] = W[index] @ A[index]
        
    return Z

In [455]:
input_vals = np.array([0.05, 0.10]).reshape((dimensions[0], 1))

W = initialise_weights(dimensions)
initial_weights = W[0]
initial_weights[0][0] = 0.15
initial_weights[0][1] = 0.2
initial_weights[1][0] = 0.25
initial_weights[1][1] = 0.30

Z = forwardpropagation(input_vals, W)

In [456]:
Z

array([array([[0.05],
       [0.1 ]]), array([[0.0275],
       [0.0425]]),
       array([[0.1213952 ],
       [0.12945374]])], dtype=object)

In [458]:
Z[1][0] + 0.35

array([0.3775])

In [461]:
Z[1][1] + 0.35

array([0.3925])